### Predictive Sport: probability of goals, shots, and more 


**Context:** 

A football game generates many events and it is very important and interesting to take into account the context in which those events were generated. This dataset should keep sports analytics enthusiasts awake for long hours as the number of questions that can be asked is huge.

Read <a href=http://crabstats.blogspot.com/>these blogs</a> to get a good understanding of soccer/football stats.


**Data description:** 

Nearly 25,000 soccer games from all leagues all over the world. The fields in the data set are: Columns A to E contains information about the league, home and away teams, date etc Columns F, G and H contain the odds for the home win, draw and away win Columns I to BQ contain the team statistics. 

Home team stats are prefixed with a "h" similarly, away team stats are prefixed with an "a". Examples include ladder position (which is a term for a rank in a group -  <a href=https://www.flashscore.com.au/football/europe/euro/standings/> here </a> an example), games played, goals conceded, away games won etc. Columns BR to CA contain final result information. That is the result, the full time result and if available, the half time score as well.

For each game there is: 
1. Statistics on the two teams, such as ladder position, win-loss history, games played 
2. Odds for home win, draw, away win (some-times is zero if odds not available) 
3. The result for that game (including the half time result if available

The dataset ranges from January 2016 to October 2017 and the statistics have been sourced from a few different websites. Odds come from BET365 and the results have been manually entered from http://www.soccerstats.com

Get more insight about the columns in the data by hovering your mouse in front of the names <a href=https://www.soccerstats.com/latest.asp?league=germany3>here</a>

**Data Location:** 
- https://www.kaggle.com/frankpac/soccerdata

---
### Instructions
Based on the scenario outlined above and the data associated with this project, you are required to perform A/B testing and Machine Learning analysis. You must also answer the quiz questions linked with this project. For convenience, the quiz questions are copied below. 

**Deadline:** Please complete the quiz, and submit all the data analysis work you carried out in this jupiter notebook via Github - see instructions below - at the very latest **by 23h59 on Thursday 8 August 2019**. 

**Why this project?**


This project is designed with the two main points in mind


We need to understand your proficiency in the important data science concepts (statistical, algorithmic, and others) and hard skills (advanced programming in python or R)
To give you an additional opportunity to add a few more data science project experience into your portfolio.  


So please put maximum effort to demonstrate your skill in this project. Answer the quiz with diligence, and perform the data analysis as best as you can. 

**Detailed instruction**:

- Git Fork/Clone the project jupyter notebooks and the corresponding data from this github: https://github.com/10acad/piq2019
The jupyter notebook “PredictiveSport.ipynb”, which contains this instruction in the first markdown cell, is for this project.
- The “data/SoccerData.xlsx” excel file is the data for this project
- Following the recommended tasks below, perform Exploratory and predictive analysis on the data.
- At the minimum you must perform an analysis that will allow you to answer some interview questions, visualize features, and produce a model to draw a reasonable conclusion. The more detailed your data analysis and clear answers to the quiz, the better it will be for your selection as 10 Academy Fellow as well as you stand a better chance to get a job interviews. Note that these notebooks and your other work at Github are critical for your Data Science Career - as they are the evidence to your skills. So even after submitting whatever you managed to do by the deadline, keep improving your model and explanations.  
- PLEASE SUBMIT WHATEVER YOU MANAGED TO DO BEFORE THE DEADLINE. WE KNOW THE TIME IS SHORT, AND IT IS FOR A PURPOSE. 
- Upload your jupyter notebook to your Github public repository. If you have forked Github link above, which is what we recommend, then you just have to do the following 
    - git add -u *  #add all modified files tracked by git 
    - git commit -m ‘submit’ 
    - git push
- Copy the Github link to your version of “PredictiveSport.ipynb” and paste it here (the 10 Academy quiz page). If you prefer, there is also a possibility to directly upload your jupyter notebook.
- If you have any questions or confusions regarding what you are expected to do in this project or how to submit, please contact community@10academy.org well before the deadline.


### Objectives:

**Exploratory analysis:**
- Understand which leagues & teams are represented in this data set - use histograms, pandas groupbys to get ideas on this 
- Explain if playing at home have a higher chance of winning? What about draws and loses?
- Explain how are predicted odds correlated to final results? 
- How many teams played more than 10 games (at home or away)?
- How are the home or away statistics distributed? Is there an imbalance in the data set?
- Be creative and explore the data set, make histograms, line (trend) plots to understand the nature of the data


**Predictive analysis:** 
- Which column has the most missing values (note that the *_odds columns are zero when no data is available)?
- Is it feasible to drop all rows with missing values? If not, try to fill missing values with an appropriate strategy.
- drop columns with greater than 50% missing values
- consider dropping teams with  
- split the data into training (60%), validation(30%) and test(10%) 
- There are 81 columns, explain which columns are highly correlated? (you may get insight on this from your exploratory analysis, plotting covariance matrices) 
- Create predictive models for football games in order to bet on football outcomes. This involves training a model for a subset of features e.g. (home,odd,ladder), and predicting a target variable e.g. win, draw, loss. Consider the following three algorithms. Comment on the relative performance if you are able to run two or three of them 
    - random forest classifier
    - xgboost 
    - neural networks
- make visualizations predicted outcomes (wins vs losses) together with the ground truth for the validation set
- make sure your model is not overfitting the training data - the statistical precision you would obtain running your model on the training data should be statistically similar to the validation data. Otherwise, the model is not generalizing (overfitting the training set)  


- Given your exploratory and predictive analysis, comment on what else you can do with the current data?